In [5]:
import numpy as np
import numpy.linalg as npl
import scipy.linalg as spl
import scipy.integrate as integrate
import matplotlib.pyplot as plt

### Résolution du problème 1D, avec conditions de Dirichlet homogènes

In [12]:
#Mesh definition
xm = 0
xM = 1

ym = 0
yM = 1
y0 = 1/2

N = 2*50

Meshm = np.meshgrid(np.linspace(xm, xM, N), np.linspace(ym, y0, int(N/2)))
MeshM = np.meshgrid(np.linspace(xm, xM, N), np.linspace(y0, yM, int(N/2)))

In [6]:
#function definition
def a(x):
    return 1

def f(x):
    return np.exp(-(x-1/2)**2)

Beta = 1    

U = np.zeros((N, N))

In [7]:
def quadrature(f, a, b, c): # A TESTER

    c1 = npl.norm(a-b, 2) # calcul des cotés
    c2 = npl.norm(a-c, 2)
    c3 = npl.norm(c-b, 2)
    p = c1 + c2 + c3      #perimetre
    aire = np.sqrt(p*(p - c1)*(p - c2)*(p - c3)) 

    p1 = f((a + b + 3*c)/5)
    p2 = f((a + 3*b + c)/5)
    p3 = f((3*a + b + c)/5)
    
    
    intval = (25/48) * (p1 + p2 + p3) * aire
    
    return intval

In [8]:
def Mat_Sys(N):
    h = (xM - xm) / (N + 1)

    wm = np.zeros(Nm+1)
    wM = np.zeros(NM+1)


    for i in range(Nm+1):
        wm[i] = (1/hm**2)*integrate.quad(lambda x :a(x),Meshm[i],Meshm[i+1])[0] #diag1
    for i in range(NM+1):
        wM[i] = (1/hM**2)*integrate.quad(lambda x :a(x),MeshM[i],MeshM[i+1])[0] #diag2
    v1 = np.concatenate((wm[1:],[Beta],wM[:-1]))                                        #Ajout des trois diago aet de la discontinuité
    v0 = np.concatenate((wm[:-1]+wm[1:],[wm[-1]+Beta],[wM[0]+Beta],wM[:-1]+wM[1:]))

    A = np.diag(-1*v1,1) + np.diag(-1*v1,-1)
    np.fill_diagonal(A, v0)

    return A

In [9]:
def phi(n, m ,x, y): # à utiliser uniquement loin de la discontinuité 

    if y<y0:
        return 
    else:
        return



    if y<y0:
        return (x > Meshm[n-1, ])*(x < Meshm[n])*(x-Meshm[n-1])/(Meshm[n]-Meshm[n-1]) + (x > Meshm[n])*(x < Meshm[n+1])*(Meshm[n+1]-x)/(Meshm[n+1]-Meshm[n])
    else:
        return (x > MeshM[n-1])*(x < MeshM[n])*(x-MeshM[n-1])/(MeshM[n]-MeshM[n-1]) + (x > MeshM[n])*(x < MeshM[n+1])*(MeshM[n+1]-x)/(MeshM[n+1]-MeshM[n])



def Sec_Mem(Nm,NM):
    fm = np.zeros(Nm+1)
    fM = np.zeros(NM+1)
    
    #i = np.linspace(1,Nm+1,Nm+1)
    #fm[:] = integrate.quad(lambda x :f(x)*phi(i[:],x),0,1)[0]    -> même pb que plus haut
    #fM[:] = integrate.quad(lambda x :f(x)*phi(i[:],x),0,1)[0]
    for i in range(Nm+1):
        fm[i] = integrate.quad(lambda x :f(x)*phi(i,x),Meshm[i],Meshm[i+1])[0]
    for i in range(NM+1):
        fM[i] = integrate.quad(lambda x :f(x)*phi(i,x),MeshM[i],MeshM[i+1])[0]
    fm[Nm] = integrate.quad(lambda x :f(x)*phi(Nm,x),0,x0)[0]
    fM[0] = integrate.quad(lambda x :f(x)*phi(0,x),x0,1)[0]

    return np.concatenate((fm,fM))


In [10]:
U[1:-1] = spl.solve(Mat_Sys(Nm,NM),Sec_Mem(Nm,NM))
plt.plot(Meshm,U[:Nm+2])
plt.plot(MeshM,U[Nm+2:])
plt.show()

NameError: name 'Nm' is not defined